In [ ]:
%matplotlib inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Listing Variables

When working with a model, it may sometimes be helpful to examine the input and output variables. Several methods are provided for this purpose.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.list_inputs
        :noindex:

    .. automethod:: openmdao.core.system.System.list_outputs
        :noindex:
```

## Example

In the following example, we create a model consisting of two instances of `ImplicitComponent`.

The implicit components are both instances of `QuadraticComp`, defined as shown here.

In [ ]:
import openmdao.api as om


class QuadraticComp(om.ImplicitComponent):
    """
    A Simple Implicit Component representing a Quadratic Equation.

    R(a, b, c, x) = ax^2 + bx + c

    Solution via Quadratic Formula:
    x = (-b + sqrt(b^2 - 4ac)) / 2a
    """

    def setup(self):
        self.add_input('a', val=1., tags=['tag_a'])
        self.add_input('b', val=1.)
        self.add_input('c', val=1.)
        self.add_output('x', val=0., tags=['tag_x'])

    def setup_partials(self):
        self.declare_partials(of='*', wrt='*')

    def apply_nonlinear(self, inputs, outputs, residuals):
        a = inputs['a']
        b = inputs['b']
        c = inputs['c']
        x = outputs['x']
        residuals['x'] = a * x ** 2 + b * x + c

    def solve_nonlinear(self, inputs, outputs):
        a = inputs['a']
        b = inputs['b']
        c = inputs['c']
        outputs['x'] = (-b + (b ** 2 - 4 * a * c) ** 0.5) / (2 * a)

These two components are placed in a `Group` with their common inputs promoted together.

In [ ]:
import openmdao.api as om

group = om.Group()

sub = group.add_subsystem('sub', om.Group(), promotes_inputs=['a', 'b', 'c'])

sub.add_subsystem('comp1', QuadraticComp(), promotes_inputs=['a', 'b', 'c'])
sub.add_subsystem('comp2', QuadraticComp(), promotes_inputs=['a', 'b', 'c'])

global prob
prob = om.Problem(model=group)
prob.setup()

prob.set_val('a', 1.)
prob.set_val('b', -4.)
prob.set_val('c', 3.)
prob.run_model()

## Usage

(list-inputs)=
### *List Inputs*

The `list_inputs()` method on a System will display all the inputs in execution order with their values. By default, the variable name and variable value are displayed. Also by default, the variables are displayed as part of the System hierarchy.

In [ ]:
prob.model.list_inputs();

### *List Outputs*

The `list_outputs()` method will display all the outputs in execution order. There are many options to this method, which we will explore below. For this example, we will only display the value in addition to the name of the output variable.

In [ ]:
prob.model.list_outputs();

### *List Implicit or Explicit Outputs*

Note that explicit and implicit outputs are listed separately. If you are only interested in seeing one or the other, you can exclude the ones you do not wish to see via the implicit and explicit arguments.

In [ ]:
prob.model.list_outputs(implicit=False);

In [ ]:
prob.model.list_outputs(explicit=False);

### *Get List via Return Value*

Both of these methods also return the information in the form of a list. You can disable the display of the information by setting the argument `out_stream` to `None` and then access the data instead via the return value.

In [ ]:
# list inputs
inputs = prob.model.list_inputs(out_stream=None)

from pprint import pprint
pprint(sorted(inputs))

### *List Names Only*

If you just want to see the names of the variables, you can disable the display of the values by setting the optional argument `val` to *False*.

In [ ]:
prob.model.list_inputs(val=False);

### *List Names and Promoted Name*

If you want the names of the variables and their promoted name within the model, you can enable the display of promoted names by setting the optional argument `prom_name` to *True*.

In [ ]:
prob.model.list_outputs(prom_name=True);

### *List Variables Filtered by Name*

You can use the `includes` and `excludes` optional arguments to filter what variables are returned from `System.list_inputs` and `System.list_outputs`. Here are some short examples showing this feature.

In [ ]:
prob.model.list_inputs(val=False, includes=['*comp2*',]);

In [ ]:
prob.model.list_outputs(val=False, excludes=['*comp2*',]);

### *List Independent Variables and Design Variables*

The `System.list_inputs` method also provides a way to determine which inputs you are ultimately responsible for setting. The [inputs report](../reports/reports_system.ipynb) achieves this in a graphical format, but this method allows it to be done programmatically.

Consider the following simple example using the Sellar model, where we intentionally have not added the variable `x` as a design variable:

In [ ]:
import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.sellar_feature import SellarMDA


model = SellarMDA()

model.add_design_var('z', lower=np.array([-10.0, 0.0]), upper=np.array([10.0, 10.0]))
# model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob = om.Problem(model)

prob.setup()
prob.final_setup();

The `is_indep_var` argument provides inputs that the user can ultimately change, though some of them maybe be overridden by the Driver as design variables:

In [ ]:
indeps = model.list_inputs(is_indep_var=True, prom_name=True)

We can also get a list of design variables using the `is_design_var` argument:

In [ ]:
desvars = model.list_inputs(is_design_var=True, prom_name=True)

Combining these two arguments will show those variables that should be set by the user and whose values will not be overridden by the Driver:

In [ ]:
nonDV_indeps = model.list_inputs(is_indep_var=True, is_design_var=False, prom_name=True)

### *List Variables Filtered by Tags*

When you add inputs and outputs to components, you can optionally set tags on the variables. These tags can then be used to filter what variables are printed and returned by the `System.list_inputs` and `System.list_outputs` methods. Each of those methods has an optional argument `tags` for that purpose.

Here is a simple example to show you how this works. Imagine that a model-builder builds a model with some set of variables they expect other non-model-builder users to vary. They want to classify the inputs into two sets: “beginner” and “advanced”. The model-builder would like to write some functions that query the model for the set of *basic* and *advanced* inputs and do some stuff with those lists (like make fancy formatted outputs or something).

In [ ]:
import openmdao.api as om

class ActuatorDiscWithTags(om.ExplicitComponent):
    """Simple wind turbine model based on actuator disc theory"""

    def setup(self):

        # Inputs
        self.add_input('a', 0.5, desc="Induced Velocity Factor", tags="advanced")
        self.add_input('Area', 10.0, units="m**2", desc="Rotor disc area", tags="basic")
        self.add_input('rho', 1.225, units="kg/m**3", desc="air density", tags="advanced")
        self.add_input('Vu', 10.0, units="m/s",
                       desc="Freestream air velocity, upstream of rotor", tags="basic")

        # Outputs
        self.add_output('Vr', 0.0, units="m/s",
                        desc="Air velocity at rotor exit plane")
        self.add_output('Vd', 0.0, units="m/s",
                        desc="Slipstream air velocity, downstream of rotor")
        self.add_output('Ct', 0.0, desc="Thrust Coefficient")
        self.add_output('thrust', 0.0, units="N",
                        desc="Thrust produced by the rotor")
        self.add_output('Cp', 0.0, desc="Power Coefficient")
        self.add_output('power', 0.0, units="W", desc="Power produced by the rotor")

    def setup_partials(self):
        self.declare_partials('Vr', ['a', 'Vu'])
        self.declare_partials('Vd', 'a')
        self.declare_partials('Ct', 'a')
        self.declare_partials('thrust', ['a', 'Area', 'rho', 'Vu'])
        self.declare_partials('Cp', 'a')
        self.declare_partials('power', ['a', 'Area', 'rho', 'Vu'])

    def compute(self, inputs, outputs):
        """ Considering the entire rotor as a single disc that extracts
        velocity uniformly from the incoming flow and converts it to
        power."""

        a = inputs['a']
        Vu = inputs['Vu']

        qA = .5 * inputs['rho'] * inputs['Area'] * Vu ** 2

        outputs['Vd'] = Vd = Vu * (1 - 2 * a)
        outputs['Vr'] = .5 * (Vu + Vd)

        outputs['Ct'] = Ct = 4 * a * (1 - a)
        outputs['thrust'] = Ct * qA

        outputs['Cp'] = Cp = Ct * (1 - a)
        outputs['power'] = Cp * qA * Vu

    def compute_partials(self, inputs, J):
        """ Jacobian of partial derivatives."""

        a = inputs['a']
        Vu = inputs['Vu']
        Area = inputs['Area']
        rho = inputs['rho']

        # pre-compute commonly needed quantities
        a_times_area = a * Area
        one_minus_a = 1.0 - a
        a_area_rho_vu = a_times_area * rho * Vu

        J['Vr', 'a'] = -Vu
        J['Vr', 'Vu'] = one_minus_a

        J['Vd', 'a'] = -2.0 * Vu

        J['Ct', 'a'] = 4.0 - 8.0 * a

        J['thrust', 'a'] = .5 * rho * Vu**2 * Area * J['Ct', 'a']
        J['thrust', 'Area'] = 2.0 * Vu**2 * a * rho * one_minus_a
        J['thrust', 'rho'] = 2.0 * a_times_area * Vu ** 2 * (one_minus_a)
        J['thrust', 'Vu'] = 4.0 * a_area_rho_vu * (one_minus_a)

        J['Cp', 'a'] = 4.0 * a * (2.0 * a - 2.0) + 4.0 * (one_minus_a)**2

        J['power', 'a'] = 2.0 * Area * Vu**3 * a * rho * (
        2.0 * a - 2.0) + 2.0 * Area * Vu**3 * rho * one_minus_a ** 2
        J['power', 'Area'] = 2.0 * Vu**3 * a * rho * one_minus_a ** 2
        J['power', 'rho'] = 2.0 * a_times_area * Vu ** 3 * (one_minus_a)**2
        J['power', 'Vu'] = 6.0 * Area * Vu**2 * a * rho * one_minus_a**2


# build the model
prob = om.Problem()
indeps = prob.model.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
indeps.add_output('a', .5, tags="advanced")
indeps.add_output('Area', 10.0, units='m**2', tags="basic")
indeps.add_output('rho', 1.225, units='kg/m**3', tags="advanced")
indeps.add_output('Vu', 10.0, units='m/s', tags="basic")

prob.model.add_subsystem('a_disk', ActuatorDiscWithTags(),
                        promotes_inputs=['a', 'Area', 'rho', 'Vu'])

# setup the optimization
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

prob.model.add_design_var('a', lower=0., upper=1.)
prob.model.add_objective('a_disk.Cp', scaler=-1)  # negated to maximize the objective

prob.setup()
prob.run_driver();

In [ ]:
prob.model.list_inputs(tags='basic', units=True, shape=True);

In [ ]:
prob.model.list_inputs(tags=['basic','advanced'], units=True, shape=True);

In [ ]:
prob.model.list_outputs(tags='basic', units=False, shape=False);

In [ ]:
prob.model.list_outputs(tags=['basic','advanced'], units=False, shape=False);

Notice that if you only have one tag, you can set the argument tags to a string. If you have more than one tag, you use a list of strings.

This example showed how to add tags when using the `add_input` and `add_output` methods. You can also add tags to `IndepVarComp` and `ExecComp` Components using code like this:

In [ ]:
comp = om.IndepVarComp('openmdao:indep_var', tags='tag1')

In [ ]:
ec = om.ExecComp('y=x+z+1.',
                 x={'val': 1.0, 'units': 'm', 'tags': 'tagx'},
                 y={'units': 'm', 'tags': ['tagy','tagm']},
                 z={'val': 2.0, 'tags': 'tagz'})

```{note}
Note that outputs of `IndepVarComp` are always tagged with `indep_var_comp`.
```

### *List Residuals Above a Tolerance*

In some cases, it might be convenient to only list variables whose residuals are above a given tolerance. The `list_outputs` method provides the optional argument `residuals_tol` for this purpose.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src68", get_code("openmdao.test_suite.components.sellar.SellarImplicitDis1"), display=False)

:::{Admonition} `SellarImplicitDis1` class definition 
:class: dropdown

{glue:}`code_src68`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src69", get_code("openmdao.test_suite.components.sellar.SellarImplicitDis2"), display=False)

:::{Admonition} `SellarImplicitDis2` class definition 
:class: dropdown

{glue:}`code_src69`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarImplicitDis1, SellarImplicitDis2
prob = om.Problem()
model = prob.model

model.add_subsystem('p1', om.IndepVarComp('x', 1.0))
model.add_subsystem('d1', SellarImplicitDis1())
model.add_subsystem('d2', SellarImplicitDis2())
model.connect('d1.y1', 'd2.y1')
model.connect('d2.y2', 'd1.y2')

model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.nonlinear_solver.options['maxiter'] = 5
model.linear_solver = om.ScipyKrylov()
model.linear_solver.precon = om.LinearBlockGS()

prob.setup()
prob.set_solver_print(level=-1)

prob.run_model()

outputs = model.list_outputs(residuals_tol=0.01, residuals=True)

In [ ]:
print(outputs)

### *List Additional Variable Metadata*

The `list_inputs()` and `list_outputs()` methods have many options to also display units, shape, bounds (lower and upper), and scaling (res, res0, and res_ref) for the variables.

In [ ]:
import openmdao.api as om

prob = om.Problem()
model = prob.model

model.add_subsystem('p1', om.IndepVarComp('x', 12.0,
                                          lower=1.0, upper=100.0,
                                          ref=1.1, ref0=2.1,
                                          units='inch',
                                          ))
model.add_subsystem('p2', om.IndepVarComp('y', 1.0,
                                          lower=2.0, upper=200.0,
                                          ref=1.2, res_ref=2.2,
                                          units='ft',
                                          ))
model.add_subsystem('comp', om.ExecComp('z=x+y',
                                        x={'val': 0.0, 'units': 'inch'},
                                        y={'val': 0.0, 'units': 'inch'},
                                        z={'val': 0.0, 'units': 'inch'}))
model.connect('p1.x', 'comp.x')
model.connect('p2.y', 'comp.y')

prob.setup()
prob.set_solver_print(level=0)
prob.run_model()

inputs = prob.model.list_inputs(units=True)

In [ ]:
print(inputs)

In [ ]:
outputs = prob.model.list_outputs(implicit=False,
                                  val=True,
                                  units=True,
                                  shape=True,
                                  bounds=True,
                                  residuals=True,
                                  scaling=True,
                                  hierarchical=False,
                                  print_arrays=False)

In [ ]:
from pprint import pprint
pprint(sorted(outputs))

In [ ]:
prob.model.list_outputs(implicit=False,
                        val=True,
                        units=True,
                        shape=True,
                        bounds=True,
                        residuals=True,
                        scaling=True,
                        hierarchical=True,
                        print_arrays=False);

### *Print Array Values*

The `list_inputs()` and `list_outputs()` methods both have a `print_arrays` option. By default, this option is set to False and only the norm of the array will appear in the tabular display. The norm value is surrounded by vertical bars to indicate that it is a norm. When the option is set to True, the complete value of the array will also be a displayed below the row.

In [ ]:
import numpy as np

import openmdao.api as om

class ArrayAdder(om.ExplicitComponent):
    """
    Just a simple component that has array inputs and outputs
    """

    def __init__(self, size):
        super().__init__()
        self.size = size

    def setup(self):
        self.add_input('x', val=np.zeros(self.size), units='inch')
        self.add_output('y', val=np.zeros(self.size), units='ft')

    def compute(self, inputs, outputs):
        outputs['y'] = inputs['x'] + 10.0

size = 30

prob = om.Problem()
prob.model.add_subsystem('des_vars', om.IndepVarComp('x', np.ones(size), units='inch'),
                         promotes=['x'])
prob.model.add_subsystem('mult', ArrayAdder(size), promotes=['x', 'y'])

prob.setup()
prob['x'] = np.arange(size)
prob.run_driver()

prob.model.list_inputs(val=True,
                       units=True,
                       hierarchical=True,
                       print_arrays=True);

In [ ]:
prob.model.list_outputs(val=True,
                        implicit=False,
                        units=True,
                        shape=True,
                        bounds=True,
                        residuals=True,
                        scaling=True,
                        hierarchical=True,
                        print_arrays=True);

You can control the format of the array values via `numpy.set_printoptions`. OpenMDAO provides the `printoptions` context manager to assist with this.

In [ ]:
from openmdao.utils.general_utils import printoptions

with printoptions(edgeitems=3, infstr='inf',
                  linewidth=75, nanstr='nan', precision=8,
                  suppress=False, threshold=1000, formatter=None):

    prob.model.list_outputs(val=True,
                            implicit=False,
                            units=True,
                            shape=True,
                            bounds=True,
                            residuals=True,
                            scaling=True,
                            hierarchical=False,
                            print_arrays=True)

### *Print Minimum or Maximum Array Values*

When working with large arrays, it can be difficult to determine how the array is interacting with the upper and lower bounds by looking through the output of the entire contents. To provide a quick visual reference, the `list_inputs()` and `list_outputs()` methods have `print_min` and `print_max` options that output columns with the minimum and maximum values of the array.

In [ ]:
prob.model.list_inputs(val=True,
                       units=True,
                       hierarchical=True,
                       print_min=True,
                       print_max=True);

In [ ]:
prob.model.list_outputs(val=True,
                        implicit=False,
                        units=True,
                        shape=True,
                        bounds=True,
                        residuals=True,
                        scaling=True,
                        hierarchical=True,
                        print_min=True,
                        print_max=True);

Note that it is normally required to run the model before `list_inputs()` and `list_outputs()` can be used. This is because the final setup that occurs just before execution determines the hierarchy and builds the data structures and connections. In some cases however, it can be useful to call these functions on a system prior to execution to assist in configuring your model. At `configure` time, basic metadata about a system’s inputs and outputs is available. 
See the documentation for the [configure](../core_features/working_with_groups/configure_method.ipynb) method for one such use case.

### *List Global Shape*

When working with [Distributed Variables](../core_features/working_with_components/distributed_components.ipynb), it may also be useful to display the global shape of a variable as well as the shape on the current processor. Note that this information is not available until after the model has been completely set up.

```{note}
This feature requires MPI, and may not be able to be run on Colab.
```

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src70", get_code("openmdao.test_suite.components.distributed_components.DistribComp"), display=False)

:::{Admonition} `DistribComp` class definition 
:class: dropdown

{glue:}`code_src70`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src71", get_code("openmdao.test_suite.components.distributed_components.Summer"), display=False)

:::{Admonition} `Summer` class definition 
:class: dropdown

{glue:}`code_src71`
:::

In [ ]:
%%px

import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.distributed_components import DistribComp, Summer
from openmdao.utils.array_utils import get_evenly_distributed_size

size = 15

prob = om.Problem()
model = prob.model

indep = model.add_subsystem("indep", om.IndepVarComp())
indep.add_output('x', np.ones(get_evenly_distributed_size(prob.comm, size)), distributed=True)
model.add_subsystem("C2", DistribComp(size=size))
model.add_subsystem("C3", Summer(size=size))

model.connect('indep.x', 'C2.invec')
model.connect('C2.outvec', 'C3.invec', src_indices=om.slicer[:])

prob.setup()
prob.final_setup()

model.C2.list_inputs(hierarchical=False, shape=True, global_shape=True, print_arrays=True);
model.C2.list_outputs(hierarchical=False, shape=True, global_shape=True, print_arrays=True);

Note that the shape of the `invec` and `outvec` variables for the distributed C2 component can be different on each processor. Use the `all_procs` argument to display on all processors

In [ ]:
%%px

prob.run_model()

model.C2.list_outputs(hierarchical=False, shape=True, global_shape=True, print_arrays=True, all_procs=True);

In [ ]:
%%px

from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['C3.sum'], -25.)

### *Listing Problem Variables*

The `Problem` class has a method `list_problem_vars` which prints out the values and metadata for design, constraint, and objective variables.

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.list_problem_vars
        :noindex:
```

You can optionally print out a variety of metadata. In this example, all the metadata is printed. The `print_arrays` option is also set to true so that full array values are printed and `min` and `max` are used so that the array's lowest and highest values are shown.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src72", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src72`
:::

In [ ]:
import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
model = prob.model
model.nonlinear_solver = om.NonlinearBlockGS()
model.linear_solver = om.ScipyKrylov()

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-9

model.add_design_var('z', lower=np.array([-10.0, 0.0]), upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.setup()
prob.run_driver();

In [ ]:
prob.list_problem_vars(print_arrays=True,
                       desvar_opts=['lower', 'upper', 'ref', 'ref0',
                                    'indices', 'adder', 'scaler',
                                    'parallel_deriv_color', 'min', 'max'],
                       cons_opts=['lower', 'upper', 'equals', 'ref', 'ref0',
                                  'indices', 'adder', 'scaler', 'linear', 'min', 'max'],
                       objs_opts=['ref', 'ref0',
                                  'indices', 'adder', 'scaler',
                                  'parallel_deriv_color',
                                  'cache_linear_solution'])